In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime
from scipy.special import gamma, factorial
import scipy.special as sc

In [2]:
def attack_free_decentralized_economic_dispatch(d_power_demand,c_wind_speed,kappa_wind_speed):
    thermal_power_station_strategy=np.zeros(shape=(53,20000)) 
    wind_power_station_strategy=np.zeros(shape=(5,20000))
    thermal_power_station_dual_variable=np.zeros(shape=(53,20000))
    wind_power_station_dual_variable=np.zeros(shape=(5,20000))
    E=np.zeros(shape=(58,58))
    thermal_power_station_cost=np.zeros(shape=(1,20000)) #火力发电成本
    wind_power_station_cost=np.zeros(shape=(1,20000)) #风力发电成本
    sum_cost=np.zeros(shape=(1,20000)) #总发电成本
    #只需要1-53号火力发电站的相关数据 
    generator_a_values= np.loadtxt("./real-data/generator_a_values.txt")#每个agent的成本函数中的超参数a
    generator_b_values= np.loadtxt("./real-data/generator_b_values.txt")#每个agent的成本函数中的超参数a
    generator_c_values= np.loadtxt("./real-data/generator_c_values.txt")#每个agent的成本函数中的超参数a
    thermal_power_station_P_max= np.loadtxt("./real-data/generator_Pmax_values.txt")#每个agent的资源量的上界
    thermal_power_station_P_min= np.loadtxt("./real-data/generator_Pmin_values.txt")#每个agent的资源量的下界
    #55 56 57 59 60号风能的成本相关数据
    wind_power_station_varphi=[1,6,1,1,6]
    wind_power_station_v_in=[3,4,5,4,5]
    wind_power_station_v_out=[25,45,25,25,45]
    wind_power_station_v_r=[13,15,16,15,16]
    wind_power_station_sigma_ue=[3,5,3,3,5]
    wind_power_station_sigma_oe=[20,30,20,20,30]
    wind_power_station_P_r=[150,160,150,150,160]
    wind_power_station_P_min=[0,0,0,0,0]
    wind_power_station_P_max=[500,300,400,300,200]
    sum_thermal_wind_power_station_strategy=np.zeros(shape=(1,20000))
    #设置迭代步长
    step_size_primal=1
    step_size_dual=1
    #设置双随机通信矩阵
    for i in range(0,58):
        for j in range(0,58):
            E[i][j]=1/58    
    #火力，风力发电策略初始化
    for i in range(0,53):
        thermal_power_station_strategy[i][0]=0
    for j in range(0,5):
        wind_power_station_strategy[j][0]=0
    
    #开始迭代
    for k in range(1,20000):
        #发电站更新计划发电量和对偶变量
        #对偶变量 先做梯度上升 再做聚合
        #梯度上升
        dual_variable_intermediate=[] #存放当前迭代中58个用户对对偶变量做完梯度下降后得到的中间值
        for i in range(0,53):
            dual_variable_intermediate.append(thermal_power_station_dual_variable[i][k-1]+step_size_dual*((1/58)*thermal_power_station_strategy[i][k-1]-(1/58)*d_power_demand))
        for j in range(0,5):
            dual_variable_intermediate.append(wind_power_station_dual_variable[j][k-1]+step_size_dual*((1/58)*wind_power_station_strategy[j][k-1]-(1/58)*d_power_demand))
        #聚合 火力发电站的对偶更新
        for i in range(0,53):
            weight_aggregation_dual_variable=0
            for j in range(0,53):
                weight_aggregation_dual_variable=weight_aggregation_dual_variable+E[i][j]*dual_variable_intermediate[j]               
            for j in range(0,5):
                weight_aggregation_dual_variable=weight_aggregation_dual_variable+E[i][j+53]*dual_variable_intermediate[j+53]
            thermal_power_station_dual_variable[i][k]=weight_aggregation_dual_variable
        #聚合 风力发电站的对偶更新
        for i in range(0,5):
            weight_aggregation_dual_variable=0
            for j in range(0,53):
                weight_aggregation_dual_variable=weight_aggregation_dual_variable+E[i+53][j]*dual_variable_intermediate[j]               
            for j in range(0,5):
                weight_aggregation_dual_variable=weight_aggregation_dual_variable+E[i+53][j+53]*dual_variable_intermediate[j+53]
            wind_power_station_dual_variable[i][k]=weight_aggregation_dual_variable
            
        #火力发电站计划发电量
        for i in range(0,53):
            thermal_power_station_strategy[i][k]=thermal_power_station_strategy[i][k-1]-step_size_primal*(2*generator_c_values[i]*thermal_power_station_strategy[i][k-1]+generator_b_values[i]+(1/58)*thermal_power_station_dual_variable[i][k-1]) 
            if thermal_power_station_strategy[i][k]<=thermal_power_station_P_min[i]:
                thermal_power_station_strategy[i][k]=thermal_power_station_P_min[i]
            elif thermal_power_station_strategy[i][k]>=thermal_power_station_P_max[i]:
                thermal_power_station_strategy[i][k]=thermal_power_station_P_max[i]
        #风力发电站计划发电量
        for i in range(0,5):
            wind_power_station_strategy[i][k]=wind_power_station_strategy[i][k-1]-step_size_primal*(wind_power_station_varphi[i]+wind_power_station_sigma_oe[i]+(wind_power_station_sigma_ue[i]+wind_power_station_sigma_oe[i])*math.exp(-(math.pow(wind_power_station_v_out[i],kappa_wind_speed))/(math.pow(c_wind_speed,kappa_wind_speed)))+(-wind_power_station_sigma_ue[i]-wind_power_station_sigma_oe[i])*math.exp(-(math.pow((wind_power_station_v_in[i]+((wind_power_station_v_r[i]-wind_power_station_v_in[i])*wind_power_station_strategy[i][k-1])/wind_power_station_P_r[i]),kappa_wind_speed))/(math.pow(c_wind_speed,kappa_wind_speed)))+(1/58)*wind_power_station_dual_variable[i][k-1])
            if wind_power_station_strategy[i][k]<=wind_power_station_P_min[i]:
                wind_power_station_strategy[i][k]=wind_power_station_P_min[i]
            elif wind_power_station_strategy[i][k]>=wind_power_station_P_max[i]:
                wind_power_station_strategy[i][k]=wind_power_station_P_max[i]
        
        #计算总计划发电
        #火力发电量
        for i in range(0,53):
            sum_thermal_wind_power_station_strategy[0][k]=sum_thermal_wind_power_station_strategy[0][k]+thermal_power_station_strategy[i][k]
        #风力发电量
        for i in range(0,5):
            sum_thermal_wind_power_station_strategy[0][k]=sum_thermal_wind_power_station_strategy[0][k]+wind_power_station_strategy[i][k]
        
        #计算总发电成本
        #火力发电成本
        for i in range(0,53):
            thermal_power_station_cost[0][k]=thermal_power_station_cost[0][k]+generator_c_values[i]*thermal_power_station_strategy[i][k]*thermal_power_station_strategy[i][k]+generator_b_values[i]*thermal_power_station_strategy[i][k]+generator_a_values[i]
        #风力发电成本
        for i in range(0,5):
            wind_power_station_cost[0][k]=wind_power_station_cost[0][k]+wind_power_station_varphi[i]*wind_power_station_strategy[i][k]+wind_power_station_sigma_ue[i]*((wind_power_station_P_r[i]-wind_power_station_strategy[i][k])*(math.exp(-(math.pow(wind_power_station_v_r[i],kappa_wind_speed))/(math.pow(c_wind_speed,kappa_wind_speed)))-math.exp(-(math.pow(wind_power_station_v_out[i],kappa_wind_speed))/(math.pow(c_wind_speed,kappa_wind_speed))))+(wind_power_station_P_r[i]*wind_power_station_v_in[i]/(wind_power_station_v_r[i]-wind_power_station_v_in[i])+wind_power_station_strategy[i][k])*(math.exp(-(math.pow(wind_power_station_v_r[i],kappa_wind_speed))/(math.pow(c_wind_speed,kappa_wind_speed)))-math.exp(-(math.pow(wind_power_station_v_in[i]+((wind_power_station_v_r[i]-wind_power_station_v_in[i])*wind_power_station_strategy[i][k])/wind_power_station_P_r[i],kappa_wind_speed))/(math.pow(c_wind_speed,kappa_wind_speed))))+(wind_power_station_P_r[i]*c_wind_speed/(wind_power_station_v_r[i]-wind_power_station_v_in[i]))*(sc.gammaincc(1+1/kappa_wind_speed,(math.pow(wind_power_station_v_in[i]+((wind_power_station_v_r[i]-wind_power_station_v_in[i])*wind_power_station_strategy[i][k])/wind_power_station_P_r[i],kappa_wind_speed))/(math.pow(c_wind_speed,kappa_wind_speed)))*sc.gamma(1+1/kappa_wind_speed)-sc.gammaincc(1+1/kappa_wind_speed,math.pow(wind_power_station_v_r[i]/c_wind_speed,kappa_wind_speed))*sc.gamma(1+1/kappa_wind_speed)))+wind_power_station_sigma_oe[i]*((wind_power_station_strategy[i][k])*(1-math.exp(-(math.pow(wind_power_station_v_in[i],kappa_wind_speed))/(math.pow(c_wind_speed,kappa_wind_speed)))+math.exp(-(math.pow(wind_power_station_v_out[i],kappa_wind_speed))/(math.pow(c_wind_speed,kappa_wind_speed))))+(wind_power_station_P_r[i]*wind_power_station_v_in[i]/(wind_power_station_v_r[i]-wind_power_station_v_in[i])+wind_power_station_strategy[i][k])*(math.exp(-(math.pow(wind_power_station_v_in[i],kappa_wind_speed))/(math.pow(c_wind_speed,kappa_wind_speed)))-math.exp(-(math.pow(wind_power_station_v_in[i]+((wind_power_station_v_r[i]-wind_power_station_v_in[i])*wind_power_station_strategy[i][k])/wind_power_station_P_r[i],kappa_wind_speed))/(math.pow(c_wind_speed,kappa_wind_speed))))+(wind_power_station_P_r[i]*c_wind_speed/(wind_power_station_v_r[i]-wind_power_station_v_in[i]))*(sc.gammaincc(1+1/kappa_wind_speed,(math.pow(wind_power_station_v_in[i]+((wind_power_station_v_r[i]-wind_power_station_v_in[i])*wind_power_station_strategy[i][k])/wind_power_station_P_r[i],kappa_wind_speed))/(math.pow(c_wind_speed,kappa_wind_speed)))*sc.gamma(1+1/kappa_wind_speed)-sc.gammaincc(1+1/kappa_wind_speed,math.pow(wind_power_station_v_in[i]/c_wind_speed,kappa_wind_speed))*sc.gamma(1+1/kappa_wind_speed)))
        #发电总成本
        sum_cost[0][k]=thermal_power_station_cost[0][k]+wind_power_station_cost[0][k]
        
    #print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    optimal_power_station_strategy_def=[]
    for i in range(0,53):
        optimal_power_station_strategy_def.append(thermal_power_station_strategy[i][19999])
    for i in range(0,5):
        optimal_power_station_strategy_def.append(wind_power_station_strategy[i][19999])
        
    optimal_dual_variable_def=[]
    for i in range(0,53):
        optimal_dual_variable_def.append(thermal_power_station_dual_variable[i][19999])
    for i in range(0,5):
        optimal_dual_variable_def.append(wind_power_station_dual_variable[i][19999])
     
    return sum_cost[0][19999], optimal_power_station_strategy_def, optimal_dual_variable_def

In [3]:
#定义变量 
d_power_demand_list=np.loadtxt("./data/d_power_demand_normal_distribution_(100,10).txt") #时变的需求侧用电需求  
c_wind_speed_list= np.loadtxt("./real-data/weibull_scale_c.txt")      
kappa_wind_speed_list= np.loadtxt("./real-data/weibull_shape_wk.txt")  
optimal_sum_cost=np.zeros(shape=(1,1000)) #最优的发电成本
optimal_power_strategy=np.zeros(shape=(1000,58)) #最优的发电策略
optimal_dual_variable=np.zeros(shape=(1000,58)) #最优的对偶变量

In [4]:
for i in range(0,1000):
    d_power_demand=d_power_demand_list[i]
    c_wind_speed=c_wind_speed_list[i]
    kappa_wind_speed=kappa_wind_speed_list[i]
    optimal_sum_cost[0][i], optimal_power_strategy[i], optimal_dual_variable[i]=attack_free_decentralized_economic_dispatch(d_power_demand,c_wind_speed,kappa_wind_speed)
    if (i+1)%100==0:
        print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])   

2024-07-22 22:41:08.011
2024-07-22 23:28:09.794
2024-07-23 00:16:48.700
2024-07-23 01:05:04.131
2024-07-23 01:52:46.946
2024-07-23 02:40:21.124
2024-07-23 03:28:04.029
2024-07-23 04:15:31.672
2024-07-23 05:04:15.967
2024-07-23 05:51:43.760


In [5]:
np.savetxt('./data/00_optimal_sum_cost_varying_D_c_kappa_normal_distribution_(100,10).txt',optimal_sum_cost,fmt='%0.8f')
np.savetxt('./data/00_optimal_power_strategy_varying_D_c_kappa_normal_distribution_(100,10).txt',optimal_power_strategy,fmt='%0.8f')
np.savetxt('./data/00_optimal_dual_variable_varying_D_c_kappa_normal_distribution_(100,10).txt',optimal_dual_variable,fmt='%0.8f')

In [6]:
optimal_sum_cost

array([[ 65543.98059701,  72892.45079015,  78857.90677404,
         80008.78110126,  96053.09746193,  69685.12235001,
        101740.03446836,  83376.20535737,  92348.00692271,
         69733.49831437,  95276.92776693,  96538.29755782,
        106952.17569108, 100343.19782329,  89891.99719387,
        101021.50439709,  89505.95060909,  77807.40989555,
         92237.23639105,  69379.35476053,  80297.73439211,
        122180.51181307,  77186.59773336,  89155.92057575,
         91862.29708089,  80637.45021431,  87094.85547228,
        100173.57789209,  92920.77541279,  92163.53015899,
         97416.13608336, 105650.9730086 ,  92252.67076772,
         76887.86886843,  95579.0023879 ,  62618.15484397,
        101378.66715378,  96850.29984625,  83109.81766992,
         79712.67174791,  94336.63936815,  93374.35358398,
         92417.22515416,  82858.15996752, 118295.70994493,
         97025.33461261,  92101.4441401 ,  87518.30376013,
        100590.89630901, 104559.68831341,  83544.9917146

In [7]:
optimal_dual_variable

array([[ -936.25774235,  -936.25774235,  -936.25774235, ...,
         -936.25774235,  -936.25774235,  -936.25774235],
       [ -983.3626739 ,  -983.3626739 ,  -983.3626739 , ...,
         -983.3626739 ,  -983.3626739 ,  -983.3626739 ],
       [-1026.67134024, -1026.67134024, -1026.67134024, ...,
        -1026.67134024, -1026.67134024, -1026.67134024],
       ...,
       [-1093.36554309, -1093.36554309, -1093.36554309, ...,
        -1093.36554309, -1093.36554309, -1093.36554309],
       [-1150.63525501, -1150.63525501, -1150.63525501, ...,
        -1150.63525501, -1150.63525501, -1150.63525501],
       [ -963.5023217 ,  -963.5023217 ,  -963.5023217 , ...,
         -963.5023217 ,  -963.5023217 ,  -963.5023217 ]])

In [8]:
optimal_power_strategy[0]

array([  5.        ,   5.        ,   5.        , 150.        ,
       149.64941923,  10.        ,  25.        ,   5.        ,
         5.        , 149.64941923, 350.        ,   8.        ,
         8.        ,  25.        ,   8.        ,  25.        ,
         8.        ,   8.        ,  25.        , 250.        ,
       250.        ,  25.        ,  25.        , 200.        ,
       200.        ,  25.        , 368.42657546, 368.42657546,
       149.64941923,  30.        ,  10.        ,   5.        ,
         5.        ,  25.        ,  25.        , 150.        ,
        25.        ,  10.        , 300.        , 149.64941923,
         8.        ,  20.        , 149.64941923, 149.64941923,
       149.64941923,   8.        ,  25.        ,  25.        ,
         8.        ,  25.        ,  25.        ,  25.        ,
        25.        , 104.26930929,  39.10216962,  67.5175539 ,
        81.15391753,  24.55671508])